<a href="https://colab.research.google.com/github/aetev/Hearth-Stone-Python-Simulator/blob/main/card_enc.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [1]:
from google.colab import drive
drive.mount('/content/drive')

Mounted at /content/drive


In [73]:
import json
import tensorflow as tf
import re
from tensorflow.keras.layers import Embedding
# Assuming AtomicCards.json is in your Google Drive, adjust the path if necessary
file_path = '/content/drive/MyDrive/MTGdata/AtomicCards.json'


with open(file_path, 'r') as f:
  data = json.load(f)['data']

In [6]:
def filter_legal_cards(data):
    legal_commander_data = {}
    for card_name, card_data in data.items():
        try:
            if card_data[0]['legalities']['commander'] == 'Legal':
                legal_commander_data[card_name] = card_data[0] # Store only the first element of card_data
        except (KeyError, IndexError):
            pass  # Or print a message, log it, etc.
    return legal_commander_data

filtered_data = filter_legal_cards(data)


In [35]:
items = list(filtered_data.items())

second_key, second_value = items[11]

text = second_value['text']
print(text)

Up to two target creatures each get +1/+0 and gain first strike until end of turn. (They deal combat damage before creatures without first strike.)
Madness {1}{R} (If you discard this card, discard it into exile. When you do, cast it for its madness cost or put it into your graveyard.)


In [94]:
class TextCleaningLayer(tf.keras.layers.Layer):
    def __init__(self,sentence_list, **kwargs):
        super(TextCleaningLayer, self).__init__(**kwargs)
        self.vectorizer = TextVectorization(standardize=None, ngrams=2, output_mode='int',output_sequence_length=150)
        self.vectorizer.adapt(cleaned_text(sentence_list))
        self.embedding_layer = Embedding(input_dim=len(self.vectorizer.get_vocabulary()), output_dim=128)

    def clean_text(self, text):
        # Convert input to string tensor if it's not already
        if not isinstance(text, tf.Tensor):
            text = tf.convert_to_tensor(text, dtype=tf.string)

        cleaned_text = tf.strings.lower(text)
        cleaned_text = tf.strings.regex_replace(cleaned_text, '\n', '')
        cleaned_text = tf.strings.regex_replace(cleaned_text, '\.', ' ')
        cleaned_text = tf.strings.regex_replace(cleaned_text, '\{|\}', ' ')
        cleaned_text = tf.strings.regex_replace(cleaned_text, '\([^)]*\)', '')
        cleaned_text = tf.strings.regex_replace(cleaned_text, '[^\w\s+/\-]', '')
        cleaned_text = tf.strings.regex_replace(cleaned_text, '\s+', ' ')
        return cleaned_text

    def call(self, inputs):
        cleaned_text = self.clean_text(inputs)
        vectorized_text = self.vectorizer(cleaned_text)
        embedded_text = self.embedding_layer(vectorized_text)

        return embedded_text

# Example usage with a batch of texts
batch_texts = tf.constant([text, "hello world test", "Another example"])
cleaned_text_layer = TextCleaningLayer(batch_texts)
cleaned_batch = cleaned_text_layer.call(batch_texts)
cleaned_batch


<tf.Tensor: shape=(3, 150, 128), dtype=float32, numpy=
array([[[ 0.03442221, -0.04857769,  0.02555412, ...,  0.01412399,
         -0.02930635,  0.04968167],
        [ 0.01911649, -0.04283419,  0.03621092, ...,  0.0102464 ,
          0.01643336, -0.01412378],
        [ 0.01949389,  0.0229933 , -0.0164119 , ..., -0.04278576,
          0.02948723,  0.01850884],
        ...,
        [ 0.01764556, -0.01266303,  0.02155865, ...,  0.02600854,
         -0.03756747, -0.02224294],
        [ 0.01764556, -0.01266303,  0.02155865, ...,  0.02600854,
         -0.03756747, -0.02224294],
        [ 0.01764556, -0.01266303,  0.02155865, ...,  0.02600854,
         -0.03756747, -0.02224294]],

       [[ 0.04492183,  0.03616545, -0.03999305, ...,  0.02049213,
          0.03244871,  0.01279643],
        [-0.03509737,  0.02589252,  0.00741037, ...,  0.02265997,
         -0.02332602,  0.00409671],
        [ 0.03693782,  0.04974368, -0.04409073, ..., -0.02781264,
          0.01214187, -0.02115973],
        ...,

In [ ]:


# Assuming 'vectorized_text' is defined from the previous code

# Define the embedding dimension
embedding_dim = 128  # You can adjust this value

# Create the embedding layer
embedding_layer = Embedding(input_dim=len(vectorizer.get_vocabulary()), output_dim=embedding_dim)

# Embed the vectorized text
embedded_text = embedding_layer(vectorized_text)

# Print the shape of the embedded text
print("Shape of embedded text:", embedded_text.shape)

# Now 'embedded_text' contains the embedded word sequences.
embedded_text
